## Data Preprocessing

#### Import Libraries

In [2]:
import pandas as pd
import numpy as np

#### Load Dataset

In [3]:
df = pd.read_csv("/Users/hpourmand/Desktop/Retail/Superstore Sales Dataset.csv")

#### Convert Date Columns to DateTime Format

In [5]:
df['Order Date'] = pd.to_datetime(df['Order Date'], format='mixed', dayfirst=True, errors='coerce')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='mixed', dayfirst=True, errors='coerce')

#### Handle Missing Values in 'Postal Code' Column

In [7]:
df['Postal Code'].fillna('Unknown', inplace=True)
df['Postal Code'].isnull().sum()

0

#### Feature Engineering with Dates 

In [8]:
df['Order Year'] = df['Order Date'].dt.year
df['Order Month'] = df['Order Date'].dt.month
df['Order Day'] = df['Order Date'].dt.day
df['Order Weekday'] = df['Order Date'].dt.weekday

#### Calculate Order Fulfillment Time (in days) for order fulfillment analysis

In [9]:
df['Fulfillment Time (Days)'] = (df['Ship Date'] - df['Order Date']).dt.days

#### Encoding Categorical Variables

In [10]:
df = pd.get_dummies(df, columns=['Ship Mode', 'Segment', 'Region'], drop_first=True)

#### Aggregate Sales Data by Different Time Frames (for Time Series Analysis)

In [12]:
monthly_sales = df.resample('M', on='Order Date')['Sales'].sum().reset_index()
monthly_sales

/var/folders/z8/rs29j2g93hb0w4pdz9_5pl400000gn/T/ipykernel_2155/1255354998.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_sales = df.resample('M', on='Order Date')['Sales'].sum().reset_index()


,Order Date,Sales
0,2015-01-31,14205.7070
1,2015-02-28,4519.8920
2,2015-03-31,55205.7970
3,2015-04-30,27906.8550
4,2015-05-31,23644.3030
5,2015-06-30,34322.9356
6,2015-07-31,33781.5430
7,2015-08-31,27117.5365
8,2015-09-30,81623.5268
9,2015-10-31,31453.3930


#### Customer Segmentation - RFM Features

##### Recency: Days since last purchase, Frequency: Total orders per customer, Monetary: Total spending per customer

In [14]:
rfm = df.groupby('Customer ID').agg({
    'Order Date': lambda x: (df['Order Date'].max() - x.max()).days,
    'Order ID': 'nunique',  # Frequency
    'Sales': 'sum'          # Monetary
}).reset_index()

# Rename columns for RFM
rfm.columns = ['Customer ID', 'Recency', 'Frequency', 'Monetary']

#### Create Product-Level Aggregates (for Product Recommendation or Inventory)

In [15]:
product_agg = df.groupby(['Product ID', 'Product Name']).agg({
    'Sales': 'sum',
    'Order ID': 'count'  # Assuming each order line represents one unit sold
}).reset_index()

# Rename columns for clarity
product_agg.columns = ['Product ID', 'Product Name', 'Total Sales', 'Total Units Sold']


#### Create Additional Time-Based Features for Forecasting

In [17]:
df['Is_Holiday_Season'] = df['Order Month'].apply(lambda x: 1 if x in [11, 12] else 0)  # Mark November and December as holiday season

In [18]:
print("Preprocessed DataFrame:\n", df.head())
print("\nMonthly Sales Data:\n", monthly_sales.head())
print("\nCustomer RFM Data:\n", rfm.head())
print("\nProduct Aggregates:\n", product_agg.head())

Preprocessed DataFrame:
    Row ID        Order ID Order Date  Ship Date Customer ID    Customer Name  \
0       1  CA-2017-152156 2017-11-08 2017-11-11    CG-12520      Claire Gute   
1       2  CA-2017-152156 2017-11-08 2017-11-11    CG-12520      Claire Gute   
2       3  CA-2017-138688 2017-06-12 2017-06-16    DV-13045  Darrin Van Huff   
3       4  US-2016-108966 2016-10-11 2016-10-18    SO-20335   Sean O'Donnell   
4       5  US-2016-108966 2016-10-11 2016-10-18    SO-20335   Sean O'Donnell   

         Country             City       State Postal Code  ...  \
0  United States        Henderson    Kentucky     42420.0  ...   
1  United States        Henderson    Kentucky     42420.0  ...   
2  United States      Los Angeles  California     90036.0  ...   
3  United States  Fort Lauderdale     Florida     33311.0  ...   
4  United States  Fort Lauderdale     Florida     33311.0  ...   

  Fulfillment Time (Days) Ship Mode_Same Day Ship Mode_Second Class  \
0                       3 

#### Save cleaned DataFrame to CSV

In [20]:
output_path = '/Users/hpourmand/Desktop/Retail/PreparedDataset.csv'
df.to_csv(output_path, index=False)